In [ ]:
%%bash
pip install -r ../../requirements.txt


### Objective
Run the dual-head multi-task transformer experiment for joint aspect and sentiment prediction.


In [ ]:
%%bash
set -e
python -m src.experiments --noise 0.3 --hierarchical --aspect-weight 1.2


Omit `--hierarchical` or adjust `--noise`/`--aspect-weight` to compare variants or simulate different aspect corruption rates.
